# Heyawake

Rules can be found [here](https://www.janko.at/Raetsel/Heyawake/).

- Blacken some cells of the grid [paper] resp. color the gray cells of the grid black or white [interactively].
- A number in a region indicates how many cells in this region must be blackened. In regions without a number any amount of cells may be blackened (maybe none).
- Black cells may not be orthogonally adjacent (diagonally is allowed).
- Stripes of adjacent white cells may not span across more than one region border horizontally or vertically.
- All white cells must form a single orthogonally contiguous area.



In [1]:
import gurobipy as gp
from collections import deque
from itertools import chain
# ===== vsualzie ===== 
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
from matplotlib.pyplot import MultipleLocator


In [28]:
def readGrid(path):
    with open(f"../assets/data/Heyawake/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n), res

if __name__ == "__main__":
    m, n, grid = readGrid("Hard_14x24")
    num_grid, maze_grid = grid[:m], grid[m:]
    for g in num_grid:
        print(g)
    for g in maze_grid:
        print(g)

['2', '-', '-', '-', '-', '-', '3', '-', '2', '-', '2', '-', '-', '2', '-', '-', '2', '-', '-', '-', '2', '-', '-', '-']
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
['1', '3', '-', '-', '-', '-', '-', '-', '-', '-', '3', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '2', '-', '-', '-', '-', '-', '-', '-', '2', '-', '-', '-', '-', '-']
['-', '-', '-', '-', '-', '-', '-', '-', '4', '-', '-', '-', '1', '-', '-', '-', '-', '-', '2', '-', '-', '-', '1', '-']
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
['-', '-', '-', '-', '-', '-', '-', '-', '2', '-', '-', '3', '-', '-', '-', '-', '-', '1', '-', '2', '-', '-', '-', '-']
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
['-', '-', '-', '-', '-', '-', '

In [30]:
def has_two_or_fewer_unique_elements(lst):
    # 使用set去重并检查不同元素的数量是否小于等于2
    return len(set(lst)) <= 2

def HeyawakeSolver(m, n, grid):
    num_grid, maze_grid = grid[:m], grid[m:]
    
    Heyawake = gp.Model('Heyawake')
    Heyawake.modelSense = gp.GRB.MINIMIZE
    Heyawake.Params.lazyConstraints = 1
    Heyawake.update()

    x = dict() # 存储决策变量
    for i in range(m):
        for j in range(n):
            x[i, j] = Heyawake.addVar(
                vtype = gp.GRB.BINARY,
                obj = 1,
                name = f"x[{i},{j}]"
            ) # 声明决策变量
    
    cell_coord = dict()
    # 记录每个宫对应的坐标的情况
    for i in range(m):
        for j in range(n):
            if maze_grid[i][j] not in cell_coord:
                cell_coord[maze_grid[i][j]] = [[i, j]]
            else:
                cell_coord[maze_grid[i][j]].append([i, j])
    
    for i in range(m):
        for j in range(n):
            if num_grid[i][j] not in ['-']:
                cur_cell = maze_grid[i][j]
                cur_cell_pos = cell_coord[cur_cell]
                Heyawake.addConstr(gp.quicksum(x[subx, suby] for (subx, suby) in cur_cell_pos) == int(num_grid[i][j]), name = f'cell_{cur_cell}')
    
    for i in range(m):
        for j in range(n):
            for k in range(j + 1, n + 1):
                if not has_two_or_fewer_unique_elements(maze_grid[i][j : k]):
                    Heyawake.addConstr(gp.quicksum(x[i, g] for g in range(j, k) ) >= 1, name = f'row_{i}_{j}_{k}')
    
    for j in range(n):
        for i in range(m):
            for k in range(i + 1, m + 1):
                temp = [maze_grid[g][j] for g in range(i, k)]
                if not has_two_or_fewer_unique_elements(temp):
                    Heyawake.addConstr(gp.quicksum(x[g, j] for g in range(i, k) ) >= 1, name = f'col_{i}_{j}_{k}')

    for i in range(m):
        for j in range(n):
            if j + 1 < n:
                Heyawake.addConstr(gp.quicksum([x[i,j] , x[i, j + 1]]) <= 1, name = f"SMALL1_{i}_{j}_1")
            if i + 1 < m:
                Heyawake.addConstr(gp.quicksum([x[i,j] , x[i + 1, j]]) <= 1, name = f"SMALL1_{i}_{j}_2")
    
    Heyawake.setObjective(gp.quicksum(x[i, j] for i in range(m) for j in range(n)), gp.GRB.MINIMIZE)
    
    def border_elim(model, where):
        if (where == gp.GRB.Callback.MIPSOL):
            x_sol = model.cbGetSolution(model._x)
            curr_grid = [[0] * n for _ in range(m)]
            for i in range(m):
                for j in range(n):
                    if int(x_sol[i, j]) == 0:
                        curr_grid[i][j] = 1
                        # 所有白色格子都被涂白了!

            def find_surrounding_black_cells(grid):
                rows, cols = len(grid), len(grid[0])
                visited = set()
                directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, down, left, right movements
                total_white = sum(chain.from_iterable(grid))
                # Find the first unvisited white cell
                def find_unvisited_white():
                    for i in range(rows):
                        for j in range(cols):
                            if grid[i][j] == 1 and (i, j) not in visited:
                                return (i, j)
                    return None
                borders_cut = [] # record cells that formulate the border: list(tuple)
                cells_inside_borders = [] # record all cells inside the border: list(tuple)
                cnt_white = 0
                while len(visited) < total_white:
                    start = find_unvisited_white()
                    if not start:
                        break
                    # Use BFS to find all connected white cells and record the surrounding black cells
                    queue = deque([start])
                    surrounding_black_cells = set()  # Use a set to avoid duplicates
                    current_inside_cells = [] # record cells in current border
                    while queue:
                        x, y = queue.popleft()
                        if (x, y) in visited:
                            continue
                        visited.add((x, y))
                        current_inside_cells.append((x, y))
                        cnt_white += 1
                        # Check all four adjacent directions
                        for dx, dy in directions:
                            nx, ny = x + dx, y + dy
                            if 0 <= nx < rows and 0 <= ny < cols:
                                if grid[nx][ny] == 1 and (nx, ny) not in visited:
                                    queue.append((nx, ny))
                                elif grid[nx][ny] == 0:
                                    surrounding_black_cells.add((nx, ny))
                    borders_cut.append(list(surrounding_black_cells))
                    cells_inside_borders.append(current_inside_cells)
                    
                assert len(borders_cut) == len(cells_inside_borders)

                if cnt_white == sum(chain.from_iterable(grid)) and len(borders_cut) == 1:
                    return [], []
                # borders_cut.sort(key = lambda x: len(x), reverse = True)
                return borders_cut, cells_inside_borders

            borders_cut, cells_inside_borders = find_surrounding_black_cells(curr_grid)

            for borders, cells_inside in zip(borders_cut, cells_inside_borders):
                if len(cells_inside) < m * n // 2:
                    for (cell_x, cell_y) in cells_inside:
                        model.cbLazy(gp.quicksum(model._x[subx, suby] for (subx, suby) in borders ) <= len(borders) - 1 + model._x[cell_x, cell_y])
                    
                
    
    Heyawake._x = x
    Heyawake.optimize(border_elim)
    # Creek.computeIIS()
    # Creek.write("Creek.ilp")
    result = [["."] * n for _ in range(m)]
    for i in range(m):
        for j in range(n):
            if x[i, j].x > 1e-6:
                print(f"X", end = " ")
                result[i][j] = "X"
            else:
                print(f".", end = " ")
        print()
    print()
    return result

if __name__ == "__main__":
    m, n, grid = readGrid("Hard_14x24")
    result = HeyawakeSolver(m, n, grid)
    

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 4821 rows, 336 columns and 43745 nonzeros
Model fingerprint: 0xcd99f102
Variable types: 0 continuous, 336 integer (336 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 4126 rows and 64 columns
Presolve time: 0.03s
Presolved: 695 rows, 272 columns, 1836 nonzeros
Variable types: 0 continuous, 272 integer (272 binary)

Root relaxation: objective 9.800000e+01, 331 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   98.00000    0   45          -   98.0000